In [1]:
import sys
sys.path.append('learn2learn/') # the version on pypi doesn't have LightningMAML ==> doing submodule approach
                                # its a little gross--> hopefully this can be change in the near future
%config Completer.use_jedi = False

In [2]:
from datasets import load_abundance_data, get_shared_taxa_dfs
from datasets import MicroDataset, Dataset
from Metalearning.MAML import tune_MAML_FFNN
import torch.nn.functional as F
import os
import pandas as pd
import numpy as np
dfs = load_abundance_data()
all_datasets = get_shared_taxa_dfs(dfs)

In [3]:
def tune_across_shared_datasets(dataset_list,
                                model_name,
                                tuning_func,
                                path, 
                                is_marker):
    if os.path.isfile(path):
        raise(ValueError('Will not run function and potentially overwrite important results! Either delete or move the file first'))
    
    # initialize csv to store results in
    results=pd.DataFrame(columns =  ['Model Type', 'Data Type', 'Dataset', 'hyperparams', 'AUC'])
    results.to_csv(path, index=False)
    # function to loop the tuning functions across each dataset
    for i in range(len(dataset_list)):
        datas = dataset_list[i:] + dataset_list[:i]
        print(datas[0].index[0])
        results=tuning_func(datas, 
                            is_marker=is_marker)
        results = pd.DataFrame([a if type(a)==str 
                                 else [b for c,b in a.items()] if type(a) == dict 
                                 else a[0]['objective'] if type(a)==tuple
                                 else a
                                 for a in results]).T
        results.columns=['Model Type', 'Data Type', 'Dataset', 'hyperparams', 'AUC']
        pd.concat( [pd.read_csv(path),results], axis=0 ).to_csv(path, index=False)
        print('Dataset #'+str(i)+' complete!')

In [4]:
names = ['Obesity', 'Colorectal', 'IBD', 'EW-T2D','C-T2D', 'Cirrhosis']
#creating different groups for param sharing
groups = {'EU': ['Obesity', 'Colorectal', 'IBD', 'EW-T2D'],
          'CH':['C-T2D', 'Cirrhosis'], 
          'T2D':['EW-T2D','C-T2D'],
          'Colon':['Colorectal','IBD']
         }

In [ ]:
%time
i = 0
mod = 'FFNN'
for g in groups:
    subset_names = groups[g]
    ds = [all_datasets[s] for s in subset_names]
    path = 'MAML_results/' + g + '_MAML_' + mod + '.csv'
    tune_across_shared_datasets(ds,
                                mod,
                                tune_MAML_FFNN,
                                path, 
                                is_marker=False
                                 )


[INFO 04-17 22:24:25] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-17 22:24:25] ax.service.managed_loop: Started full optimization with 15 steps.
[INFO 04-17 22:24:25] ax.service.managed_loop: Running optimization trial 1...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
830


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
[INFO 04-17 22:25:02] ax.service.managed_loop: Running optimization trial 2...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
[INFO 04-17 22:26:29] ax.service.managed_loop: Running optimization trial 3...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
[INFO 04-17 22:29:58] ax.service.managed_loop: Running optimization trial 4...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
